Monthly claim count trend over last 12 months

In [0]:
SELECT  
    TO_CHAR(claim_date::date, 'YYYY-MM') AS claim_month,
    COUNT(claim_id) AS claim_count,
    SUM(claim_amount) AS total_billed,
    SUM(paid_amount) AS total_paid,
    COUNT(DISTINCT patient_id) AS unique_patients,
    COUNT(DISTINCT provider_id) AS unique_providers,
    ROUND(AVG(claim_amount), 2) AS avg_claim_amount
FROM healthcare_cp.hcp.claims_data
WHERE claim_date >= CURRENT_DATE - INTERVAL '12 months'
GROUP BY TO_CHAR(claim_date::date, 'YYYY-MM')
ORDER BY claim_month;

Seasonal spikes in specific procedures (flu, orthopedics, etc.)

In [0]:
WITH seasonal_analysis AS (
    SELECT 
        procedure_code,
        EXTRACT(MONTH FROM service_start_date::date) AS service_month,
        CASE 
            WHEN EXTRACT(MONTH FROM service_start_date::date) IN (12, 1, 2) THEN 'Winter'
            WHEN EXTRACT(MONTH FROM service_start_date::date) IN (3, 4, 5) THEN 'Spring'
            WHEN EXTRACT(MONTH FROM service_start_date::date) IN (6, 7, 8) THEN 'Summer'
            WHEN EXTRACT(MONTH FROM service_start_date::date) IN (9, 10, 11) THEN 'Fall'
        END AS season,
        COUNT(claim_id) AS procedure_count
    FROM healthcare_cp.hcp.claims_data
    WHERE service_start_date >= CURRENT_DATE - INTERVAL '24 months'
    AND procedure_code IS NOT NULL
    GROUP BY procedure_code, service_month, season
),
procedure_totals AS (
    SELECT 
        procedure_code,
        SUM(procedure_count) AS total_procedures
    FROM seasonal_analysis
    GROUP BY procedure_code
    HAVING SUM(procedure_count) >= 100  -- Focus on common procedures
)
SELECT 
    sa.procedure_code,
    pt.total_procedures,
    sa.season,
    SUM(sa.procedure_count) AS seasonal_count,
    ROUND((SUM(sa.procedure_count) * 100.0 / pt.total_procedures), 2) AS seasonal_percentage,
    ROUND(AVG(SUM(sa.procedure_count)) OVER (PARTITION BY sa.procedure_code), 2) AS avg_seasonal_count
FROM seasonal_analysis sa
JOIN procedure_totals pt ON sa.procedure_code = pt.procedure_code
GROUP BY sa.procedure_code, pt.total_procedures, sa.season
ORDER BY sa.procedure_code, seasonal_percentage DESC;

Average time between service_end_date and claim_date (delay in submission)

In [0]:
SELECT 
    submission_mode,
    COUNT(claim_id) AS total_claims,
    ROUND(AVG(claim_date::date - service_end_date::date), 2) AS avg_submission_delay_days,
    ROUND(STDDEV(claim_date::date - service_end_date::date), 2) AS std_dev_delay,
    MIN(claim_date::date - service_end_date::date) AS min_delay_days,
    MAX(claim_date::date - service_end_date::date) AS max_delay_days,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY claim_date::date - service_end_date::date) AS median_delay_days,
    COUNT(CASE WHEN (claim_date::date - service_end_date::date) > 30 THEN 1 END) AS delays_over_30_days
FROM healthcare_cp.hcp.claims_data
WHERE service_end_date IS NOT NULL 
  AND claim_date IS NOT NULL
  AND submission_mode IS NOT NULL
GROUP BY submission_mode
ORDER BY avg_submission_delay_days;

Mode of submission vs claim approval rate (Online vs EDI vs Paper)

In [0]:
SELECT 
    submission_mode,
    COUNT(claim_id) AS total_claims,
    COUNT(CASE WHEN claim_status = 'Approved' THEN 1 END) AS approved_claims,
    COUNT(CASE WHEN claim_status = 'Denied' THEN 1 END) AS denied_claims,
    COUNT(CASE WHEN claim_status = 'Pending' THEN 1 END) AS pending_claims,
    ROUND((COUNT(CASE WHEN claim_status = 'Approved' THEN 1 END) * 100.0 / COUNT(claim_id)), 2) AS approval_rate_pct,
    ROUND((COUNT(CASE WHEN claim_status = 'Denied' THEN 1 END) * 100.0 / COUNT(claim_id)), 2) AS denial_rate_pct,
    ROUND(AVG(claim_date::date - service_end_date::date), 2) AS avg_submission_delay,
    ROUND(AVG(CASE WHEN claim_status = 'Approved' THEN paid_amount END), 2) AS avg_approved_payment
FROM healthcare_cp.hcp.claims_data
WHERE submission_mode IS NOT NULL
GROUP BY submission_mode
ORDER BY approval_rate_pct DESC;